In [1]:
!pip install transformers -U

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.2/10.2 MB 61.1 MB/s eta 0:00:00
  Attempting uninstall: transformers
    Found existing installation: transformers 4.50.2
    Uninstalling transformers-4.50.2:
      Successfully uninstalled transformers-4.50.2


In [2]:
import pandas as pd

data= pd.read_csv('/content/drive/MyDrive/train.csv')
data.head()

,id,comment_text,toxic,severe_toxic,obscene,threat,insult,identity_hate
0,0000997932d777bf,Explanation\nWhy the edits made under my usern...,0,0,0,0,0,0
1,000103f0d9cfb60f,D'aww! He matches this background colour I'm s...,0,0,0,0,0,0
2,000113f07ec002fd,"Hey man, I'm really not trying to edit war. It...",0,0,0,0,0,0
3,0001b41b1c6bb37e,"""\nMore\nI can't make any real suggestions on ...",0,0,0,0,0,0
4,0001d958c54c6e35,"You, sir, are my hero. Any chance you remember...",0,0,0,0,0,0


In [3]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score
from transformers import Trainer,TrainingArguments
from transformers import BertTokenizer, BertForSequenceClassification

tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertForSequenceClassification.from_pretrained('bert-base-uncased',num_labels=2)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [4]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSdpaSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e

In [5]:
model=model.to('cuda')

In [6]:
sample_data=["I am eating","I am Playing"]
tokenizer(sample_data,padding=True,truncation=True,max_length=512,return_tensors='pt')


{'input_ids': tensor([[ 101, 1045, 2572, 5983,  102],
        [ 101, 1045, 2572, 2652,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0],
        [0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1],
        [1, 1, 1, 1, 1]])}

In [33]:
data= data[0:1000]
X= data['comment_text']
y=data['toxic']

x_train,x_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

x_train_tokenized=tokenizer(list(x_train),padding=True,truncation=True,max_length=512,return_tensors='pt')
x_test_tokenized=tokenizer(list(x_test),padding=True,truncation=True,max_length=512,return_tensors='pt')


In [42]:
import torch
class Dataset(torch.utils.data.Dataset):
  def __init__(self,encodings,labels=None):
    self.encodings=encodings
    self.labels=labels

  def __getitem__(self,idx):
      item={key:torch.tensor(val[idx]) for key,val in self.encodings.items()}
      # Check if labels is not None and it's not empty instead of directly using if self.labels
      if self.labels is not None and len(self.labels) > 0:
        # Use .iloc[idx] to access by position
        item['labels']=torch.tensor(self.labels.iloc[idx])  # Access using .iloc
      return item # return the item outside the if statement. This will also cater case where labels is None

  def __len__(self):
      return len(self.encodings['input_ids'])


In [43]:
train_dataset=Dataset(x_train_tokenized,y_train)
test_dataset=Dataset(x_test_tokenized,y_test)


In [44]:
train_dataset[0]

<ipython-input-42-f59ef5790996>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item={key:torch.tensor(val[idx]) for key,val in self.encodings.items()}


{'input_ids': tensor([  101,  1000,  1027,  1027,  1037, 25684,  7559,  2005,  2017,   999,
          1027,  1027,  1996,  2613,  2166, 25684,  7559, 11082,  2149,  2022,
          1996,  3340,  1000,   102,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0,     0,
             0,     0,     0,     0,   

In [37]:
def compute_metrics(p):
  pred,labels=p
  pred=np.argmax(pred,axis=1)
  accuracy=accuracy_score(y_true=labels,y_pred=pred)
  recall=recall_score(y_true=labels,y_pred=pred)
  precision=precision_score(y_true=labels,y_pred=pred)
  f1=f1_score(y_true=labels,y_pred=pred)
  return {"accuracy":accuracy,"precision":precision,"recall":recall,"f1":f1}


In [45]:
args= TrainingArguments(
    output_dir='output',
    num_train_epochs=1,
    per_device_train_batch_size=8,
    report_to="none"
)

# trainer= Trainer(
#     model=model,
#     args=args,
#     train_dataset=train_dataset,
#     eval_dataset=test_dataset,
#     compute_metrics=compute_metrics
# )

trainer= Trainer(
    model=model,
    args=args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    compute_metrics=compute_metrics
)

In [46]:
trainer.train()

<ipython-input-42-f59ef5790996>:8: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
  item={key:torch.tensor(val[idx]) for key,val in self.encodings.items()}


Step,Training Loss


TrainOutput(global_step=100, training_loss=0.25886091232299807, metrics={'train_runtime': 79.3794, 'train_samples_per_second': 10.078, 'train_steps_per_second': 1.26, 'total_flos': 210488844288000.0, 'train_loss': 0.25886091232299807, 'epoch': 1.0})

trainer.evaluate()

In [56]:
text="That was good point"
inputs= tokenizer(text,padding=True,truncation=True,max_length=512,return_tensors='pt').to('cuda')
output = model(**inputs)
output

predictions= torch.nn.functional.softmax(output.logits,dim=-1)
predictions
predictions= predictions.cpu().detach().numpy()
predictions

array([[0.9730023 , 0.02699768]], dtype=float32)

In [57]:
trainer.save_model('toxic_model')

In [59]:
toxic_model=BertForSequenceClassification.from_pretrained('toxic_model')
toxic_model = toxic_model.to('cuda')
text="you are so toxic"#"That was a bad example"
inputs= tokenizer(text,padding=True,truncation=True,max_length=512,return_tensors='pt').to('cuda')
output = toxic_model(**inputs)
output

predictions= torch.nn.functional.softmax(output.logits,dim=-1)
predictions
predictions= predictions.cpu().detach().numpy()
predictions

array([[0.32194743, 0.67805254]], dtype=float32)